# 基于MindSpore实现语音识别
本小节介绍并展示了如何使用MindSpore实现语音识别。

## 1、实验目的
- 了解语音识别。
- 掌握如何使用MindSpore进行语音识别。

## 2、语音识别介绍
语音识别（Speech Recognition）是机器学习领域中的一个重要应用，旨在将人类语音转化为可理解和可处理的文本形式。它可以帮助计算机理解和处理人类的语言，从而实现自然语言交互、语音命令识别、语音转写等功能。

## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 处理数据集
本实验使用到了mini speech commands数据集。mini speech commands是一个用于语音识别的小型数据集，常用于学术和教育目的。它由Google团队创建，并提供给机器学习社区进行研究和开发。该数据集主要用于进行关键词识别任务，即识别特定的一组预定义关键词。这些关键词通常是日常生活中常用的命令，如"yes"、"no"、"up"、"down"、"go"、"stop"等。mini_speech_commands数据集包含了数千个短语音片段，每个片段持续约1秒，并由多个不同说话者录制。

In [1]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")

使用语音数据集进行音频文件的特征提取，得到数据集的特征矩阵，用以进行分类处理。

In [2]:
import numpy as np
import scipy.io.wavfile as wav

# 读取文件并进行特征提取的函数
def get_spectrogram(file_path):
    fs, waveform = wav.read(file_path)
    # 声谱的矩阵大小[124,129]
    # spectrogram = np.zeros([124, 129]).astype(np.float32)
    spectrogram = np.zeros([124, 129]).astype(np.float32)
    # 边距
    zero_padding = np.zeros([16000 - waveform.shape[0]], dtype=np.float32)
    waveform = waveform.astype(np.float32)
    # 扩充到16000
    equal_length = np.concatenate([waveform, zero_padding])
    # 生成0-254每个整数
    x = np.linspace(0, 254, 255, dtype=np.int32)
    # 在数字信号处理中，加窗是音频信号预处理重要的一步
    window = 0.54 - 0.46 * np.cos(2 * np.pi * (x) / (255 - 1))
    for i in range(124):
        # 帧头
        p_start = i * 128
        # 帧尾
        p_end = p_start + 255
        frame_data = equal_length[p_start:p_end]
        frame_data = frame_data * window
        # 离散傅里叶变化
        spectrum = np.fft.rfft(frame_data, n=256)
        # 经过修改后可以使得特征输出为[124,129]
        spectrogram[i,:] = np.abs(spectrum)
    return spectrogram

将数据集划分训练集、验证级和测试集，并将打散后的数据顺序保存下来，用于训练的文件名保存在train_file.txt中，用于测试的文件名保存在test_file.txt中。

In [3]:
import os
import random
import glob

data_dir = 'data/mini_speech_commands'

# 获取命令列表
commands = [dir_name for dir_name in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, dir_name)) and dir_name != 'README.md']

# 设置随机种子
seed = 42
random.seed(seed)

# 打乱命令顺序
random.shuffle(commands)

# 获取所有文件名
all_files = []
for command in commands:
    command_path = os.path.join(data_dir, command)
    files = glob.glob(os.path.join(command_path, '*.wav'))
    all_files.extend(files)

# 打乱文件顺序
random.shuffle(all_files)

# 划分训练、验证和测试集
train_files = all_files[:6400]
val_files = all_files[6400: 6400 + 800]
test_files = all_files[-800:]

# 保存训练和测试文件列表到文件中
train_file_path = 'train_file.txt'
test_file_path = 'test_file.txt'

with open(train_file_path, 'w', encoding='utf-8') as file1:
    for f in train_files:
        file1.write(f.replace('\\', '\\\\').replace('/', '\\\\') + '\n')

with open(test_file_path, 'w', encoding='utf-8') as file2:
    for f in test_files:
        file2.write(f.replace('\\', '\\\\').replace('/', '\\\\') + '\n')

读取特征文件。

In [4]:
# 具体识别的8个词
commands = ['yes', 'no', 'up', 'down', 'right', 'left', 'go', 'stop']
# 获取音频标签
# 读取特征文件中的数据
def get_data(file_path):
    with open(file_path, 'r', encoding='utf8') as f:
        files = f.readlines()
    # 逐行读取
    for line in files:
        line = line.strip()
        # 提取label
        data = get_spectrogram(line)
        label = line.split('\\\\')[-2]
        label_id = commands.index(label)
        # print(data,label_id,"##")
        yield np.array(data,dtype=np.float32), label_id

数据增强，并将数据进行批处理。

In [5]:
import mindspore.dataset as ds

# 意为数据集本身每一条数据都可以通过索引直接访问
ds_train = ds.GeneratorDataset(list(get_data(train_file_path)), column_names=['data', 'label'])
# 批处理,分为64批
ds_train = ds_train.batch(64)

## 5、模型构建

把音频转换为语谱图后，其实就是把语音问题转化为图像问题，图像中就包含了声音的特征，这里定义了一个CV类的网络。

In [6]:
from mindspore.nn import Conv2d
from mindspore.nn import MaxPool2d
from mindspore.nn import Cell
import mindspore.ops.operations as P
from mindspore.nn import Dense
from mindspore.nn import ReLU
from mindspore.nn import Flatten
from mindspore.nn import Dropout
from mindspore.nn import BatchNorm2d

# 实现二维卷积操作
def conv2d(in_channels, out_channels):
    return Conv2d(in_channels=in_channels, out_channels=out_channels,
                  kernel_size=3, stride=1, pad_mode='valid',
                  has_bias=True, weight_init='he_normal')               
# 池化层
def maxpool():
    return MaxPool2d(kernel_size=(2, 2), stride=(2, 2), pad_mode='valid')

# 定义网络
class Net(Cell):
    def __init__(self, batch_size):
        super(Net, self).__init__()
        # 向网络中加层
        self.batch_size = batch_size
        self.reshape = P.Reshape()
        self.resize = P.ResizeNearestNeighbor(size=(32, 32))
        self.norm = BatchNorm2d(num_features=1)
        self.conv1 = conv2d(1, 32)
        self.relu1 = ReLU()
        self.conv2 = conv2d(32,64)
        self.relu2 = ReLU()
        self.maxpool = maxpool()
        self.dropout1 = Dropout(keep_prob=0.25)
        self.flatten = Flatten()
        self.dense1 = Dense(in_channels=12544, out_channels=128)
        self.relu3 = ReLU()
        self.dropout2 = Dropout(keep_prob=0.5)
        self.dense2 = Dense(in_channels=128, out_channels=8)
    
    def construct(self, input_x):
        x = self.reshape(input_x, (self.batch_size,1, 124, 129))
        x = self.resize(x)
        x = self.norm(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu3(x)
        x = self.dropout2(x)
        x = self.dense2(x)
        return x

损失函数为SoftmaxCrossEntropyWithLogits，优化器为Adam。

In [7]:
from mindspore import Model
from mindspore.nn import Adam
from mindspore.nn import SoftmaxCrossEntropyWithLogits

# 构建网络
net = Net(batch_size=64)
# 优化器
opt = Adam(net.trainable_params(), learning_rate=0.0008,
           beta1=0.9, beta2=0.999, eps=10e-8, weight_decay=0.01)
# softmax损失函数
loss_fn = SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean')
# 利用训练数据集训练模型
model = Model(net, loss_fn, opt)  

## 6、模型训练

执行训练。

In [8]:
from mindspore.train.callback import CheckpointConfig
from mindspore.train.callback import ModelCheckpoint

# 训练10轮，中间保存一下ckpt
config = CheckpointConfig(save_checkpoint_steps=10, saved_network=net)
# 将结果保存在./checkpoint中
ckpoint_cb = ModelCheckpoint(prefix='asr', directory='./checkpoint', config=config)
# 进行模型训练
model.train(10, ds_train, ckpoint_cb)

## 7、模型预测
使用模型进行预测，并测试准确率。

In [9]:
net = Net(batch_size=1)

from mindspore import load_checkpoint
from mindspore import Tensor

# 读取训练的模型文件
ckpt_file_name = "./checkpoint/asr-10_100.ckpt"
param_dict = load_checkpoint(ckpt_file_name, net)

print("****start test****")
# 获取测试文件
batch = get_data(test_file_path) 
print(batch)
# 初始化准确率
accu = 0 
size=800

# 根据训练好的模型进行预测
for i in range(size):
    input_x, label = next(batch)
    output = net(Tensor(input_x))
    index = np.argmax(output.asnumpy())
    # 输出期望值、预测值
    print(commands[index], commands[label]) 
    if index == label:
        # 若预测成功则成功数量+1，记录预测成功的样本数量
        accu += 1      
# 准确率
print("accuracy: ", accu*1.0 / size )

****start test****
<generator object get_data at 0x000002A61D2B49C8>
go right
yes yes
yes no
down down
stop up
left left
no no
yes yes
yes left
stop stop
down down
left right
right right
yes yes
down down
no no
down down
go go
go go
no no
stop stop
stop stop
left right
right right
yes yes
left left
yes yes
yes stop
no stop
right right
left left
stop stop
yes yes
yes yes
up no
down down
down down
stop stop
yes yes
up go
go no
no no
right right
right right
down down
go go
up up
up stop
right right
go down
go down
no no
up up
stop stop
up stop
stop stop
left left
up up
go no
stop stop
right right
down down
no go
down down
yes yes
left left
go right
stop stop
yes yes
no go
yes yes
up up
stop stop
go go
left left
down down
yes yes
right right
up up
right up
no no
go down
left left
yes yes
up up
up left
stop stop
right right
down down
stop stop
yes yes
stop stop
up yes
right right
right right
yes left
stop up
yes stop
yes yes
up up
yes yes
go go
up up
left left
yes up
stop stop
right right
d

可以看到，最终测试集的准确率为0.73625，训练效果良好，基本达到语音识别分类目的。